In [ ]:
#| default_exp argument_validators

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from typing import Protocol, Any, runtime_checkable
from pymoq.core import AnyCallable

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import test_fail

# Validators

> Collection of methods to validate specific call arguments.

Goal: Evaluate whether a call like `f(1,"s")` matches any signature-pattern. A signature pattern might be defined like `f(1, str)`. This should match any call that passes the exact value one for the first argument and any object of type str in the second.

## Argument validators

We break the task down to validating a single argument. The signature of such an ArgumentValidator should look like :

In [ ]:
#| export
@runtime_checkable
class ArgumentValidator(Protocol):
    "Interface for all argument validators."
    
    @property
    def name() -> str:
        "Name of the argument in the signature"
        
    @property
    def position() -> int:
        "Position of the argument in the signature"
        
    def is_valid(self, argument: Any) -> bool:
        ...

The most flexibility can be achieved by constructing an ArgumentValidator that evaluates an arbitrary function:

In [ ]:
#| export
class ArgumentFunctionValidator:
    "Validate an argument by evaluating an arbitrary function"
    def __init__(self, func: AnyCallable[bool], name: str, position: int):
        self._func = func
        self._name = name
        self._position = position
        
    @property
    def name(self) -> str:
        return self._name
    
    @property
    def position(self) -> int:
        return self._position
        
    def is_valid(self, argument: Any) -> bool:
        return self._func(argument)
    
    def __str__(self): return f'ArgumentFunctionValidator({self.name}, {self.position})'
    def __repr__(self): return str(self)
    
assert isinstance(ArgumentFunctionValidator, ArgumentValidator), "ArgumentFunctionValidator does not implement the ArgumentValidator-Protocol"

This could now be used like:

In [ ]:
any_int = ArgumentFunctionValidator(lambda v: isinstance(v, int), "firstArgument", 0)
any_int

ArgumentFunctionValidator(firstArgument, 0)

In [ ]:
assert any_int.is_valid(1)
assert not any_int.is_valid(1.1)
assert not any_int.is_valid("string")
assert str(any_int)=='ArgumentFunctionValidator(firstArgument, 0)'

In later stages there should be convenience methods around creating such argument validators. E.g. `from_type(some_type)` for making the above easier.

### Ease of use: Construction from arguments

In [ ]:
#| export
def argument_validator_from_argument(argument: Any, name:str, position: int) -> ArgumentValidator:
    if isinstance(argument, ArgumentValidator):
        return argument
    
    if callable(argument):
        return ArgumentFunctionValidator(argument, name=name, position=position)
    else:
        return ArgumentFunctionValidator(lambda v: v==argument, name=name, position=position)

Passing a valid `ArgumentValidator` simply returns it:

In [ ]:
assert argument_validator_from_argument(any_int, any_int.name, 0) == any_int

Passing a callable constructs an `ArgumentFunctionValidator`:

In [ ]:
arg_val = argument_validator_from_argument(lambda v: isinstance(v, int), 'any_int', 0)

assert isinstance(arg_val, ArgumentValidator)
assert arg_val.is_valid(1)
assert not arg_val.is_valid("1")

Passing a non-callable assumes that the value should be compared against, i.e. it's a constant:

In [ ]:
arg_val = argument_validator_from_argument(123, name='any_int', position=0)

assert isinstance(arg_val, ArgumentValidator)
assert arg_val.is_valid(123)
assert not arg_val.is_valid(124)

## Special Validators

In [ ]:
#| export
AnyArg = lambda: lambda v: True

# Build library

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()